<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/deeplearning.ai/nlp/c1_w3_lecture_02_word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Manipulating Word Embeddings

This notebook explores how to apply linear algebra to find analogies between words manually.

In [5]:
!wget 'https://github.com/martin-fabbri/colab-notebooks/raw/master/data/nlp/word_embeddings_subset.p'

--2020-11-03 14:08:20--  https://github.com/martin-fabbri/colab-notebooks/raw/master/data/nlp/word_embeddings_subset.p
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/martin-fabbri/colab-notebooks/master/data/nlp/word_embeddings_subset.p [following]
--2020-11-03 14:08:20--  https://raw.githubusercontent.com/martin-fabbri/colab-notebooks/master/data/nlp/word_embeddings_subset.p
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 309156 (302K) [application/octet-stream]
Saving to: ‘word_embeddings_subset.p’

word_embeddings_sub 100%[===================>] 301.91K  --.-KB/s    in 0.05s   

2020-11-03 14:

In [7]:
import pandas as pd
import numpy as np
import pickle 

word_embeddings_url = 'word_embeddings_subset.p'
word_embeddings = pickle.load(open(word_embeddings_url, 'rb'))
len(word_embeddings)

243

Now that the model is loaded, we can take a look at the word representations. First, note that the `word embeddings` is a dictionary. Each word is the key to the entry, and the value is its corresponding vector presentation. Remember that square brackers allow access to an entry if the key exists. 

In [11]:
country_vector = word_embeddings['country']
print(f'vector type: {type(country_vector)}')
print(f'vector shape: {country_vector.shape}')
country_vector[:3]

vector type: <class 'numpy.ndarray'>
vector shape: (300,)


array([-0.08007812,  0.13378906,  0.14355469], dtype=float32)

It is important to note that we store each vector as a np array. It allows us to use the linear algebra operations on it.

The vectors have a a size of 300, while the vocabulary size of Google News is around 3 million words!

In [ ]:
def vec(w):
  return word_embeddings[w]

## Operating on word embeddings

Remember that understanding that data is one of the most critical steps in Data Science. Word embeddings are the result of machine learning processes and will be part of the input for further processes. These word embedding needs to be validated or at least understood because the performance of the derived model will strongly depend on its quality.